In [1]:
import numpy as np
import pandas as pd
import requests
import zipfile

# PREM Aggregation

In [56]:
# Download PREM contact matrices
url = 'https://doi.org/10.1371/journal.pcbi.1005697.s002'
r = requests.get(url, allow_redirects=True)
open("cm.zip", 'wb').write(r.content)

# Unzip PREM contact matrices
with zipfile.ZipFile('cm.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [57]:
country = "Hungary"
original_prem = pd.read_excel("contact_matrices_152_countries/MUestimates_all_locations_1.xlsx", sheet_name=country)
populacio_table = pd.read_html('https://www.ksh.hu/stadat_files/nep/hu/nep0003.html')

In [58]:
df2=populacio_table[0].tail(279//3)
df2.drop([186,278],inplace=True)
df2=df2[["Korév","2021"]]
df2["Korév"][277] = 90
df2['2021'] = df2['2021'].str.replace(" ","")
df2 = df2.astype(int)
df2.set_index("Korév", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [59]:
ksh_pop_vector=[]
for i in range(0,71,5):
  ksh_pop_vector.append(df2["2021"].loc[i:i+4].sum())
ksh_pop_vector.append(df2["2021"].loc[75:].sum())
ksh_pop_vector = np.array(ksh_pop_vector).reshape(-1, 1)

In [60]:
full_prem = original_prem.to_numpy() * ksh_pop_vector
full_prem = (full_prem + full_prem.T)/2

In [61]:
reduced_ksh_pop_vector = np.array([ksh_pop_vector[0:3,:].sum(),
                                   ksh_pop_vector[3:12,:].sum(),
                                   ksh_pop_vector[12:,:].sum()]).reshape(-1, 1)
reduced_ksh_pop_vector

array([[1417233],
       [5723488],
       [2590051]])

In [62]:
reduced_full_prem = np.array([[full_prem[0:3,0:3].sum(),full_prem[0:3,3:12].sum(),full_prem[0:3,12:].sum()],
                              [full_prem[3:12,0:3].sum(),full_prem[3:12,3:12].sum(),full_prem[3:12,12:].sum()],
                              [full_prem[12:,0:3].sum(),full_prem[12:,3:12].sum(),full_prem[12:,12:].sum()]])

In [63]:
reduced_full_prem

array([[ 8032023.78703636,  7862617.46303875,  1933589.80501905],
       [ 7862617.46303875, 67823712.36405726,  7795719.38950609],
       [ 1933589.80501905,  7795719.38950609,  5995078.60275472]])

In [64]:
reduced_prem = reduced_full_prem * (1/reduced_ksh_pop_vector)
reduced_prem

array([[ 5.66739822,  5.54786507,  1.36434151],
       [ 1.37374578, 11.85006632,  1.36205744],
       [ 0.74654507,  3.009871  ,  2.31465659]])

# Viruses

In [65]:
viruses_pop_vector = [468605, 953134, 1678211, 4087976, 1312208, 839589, 433033]
viruses_pop_vector_aggr = [sum(viruses_pop_vector[0:2]),
                           sum(viruses_pop_vector[2:4]),
                           sum(viruses_pop_vector[4:])]
viruses_pop_vector_aggr

[1421739, 5766187, 2584830]

In [66]:
viruses_p = [0.95, 0.8, 0.7, 0.5, 0.4, 0.3, 0.2]
viruses_xi = [0.333, 0.333, 0.297, 0.294, 0.292, 0.293, 0.293]
viruses_h = [0.00045, 0.00045, 0.0042, 0.0442, 0.1162, 0.2682, 0.4945]
viruses_mu = [0.2, 0.2, 0.216, 0.3, 0.582, 0.678, 0.687]

In [67]:
aggr_xi = np.array([(viruses_pop_vector[0]*viruses_xi[0]+viruses_pop_vector[1]*viruses_xi[1])/viruses_pop_vector_aggr[0],
           (viruses_pop_vector[2]*viruses_xi[2]+viruses_pop_vector[3]*viruses_xi[3])/viruses_pop_vector_aggr[1],
           (viruses_pop_vector[4]*viruses_xi[4]+viruses_pop_vector[5]*viruses_xi[5]+viruses_pop_vector[6]*viruses_xi[6])/viruses_pop_vector_aggr[2]])
aggr_xi

array([0.333     , 0.29487313, 0.29249234])

In [68]:
aggr_p = np.array([(viruses_pop_vector[0]*viruses_p[0]+viruses_pop_vector[1]*viruses_p[1])/viruses_pop_vector_aggr[0],
          (viruses_pop_vector[2]*viruses_p[2]+viruses_pop_vector[3]*viruses_p[3])/viruses_pop_vector_aggr[1],
          (viruses_pop_vector[4]*viruses_p[4]+viruses_pop_vector[5]*viruses_p[5]+viruses_pop_vector[6]*viruses_p[6])/viruses_pop_vector_aggr[2]])
aggr_p

array([0.84943998, 0.55820869, 0.33401288])

In [69]:
aggr_h = np.array([(viruses_pop_vector[0]*viruses_h[0]+viruses_pop_vector[1]*viruses_h[1])/viruses_pop_vector_aggr[0],
          (viruses_pop_vector[2]*viruses_h[2]+viruses_pop_vector[3]*viruses_h[3])/viruses_pop_vector_aggr[1],
          (viruses_pop_vector[4]*viruses_h[4]+viruses_pop_vector[5]*viruses_h[5]+viruses_pop_vector[6]*viruses_h[6])/viruses_pop_vector_aggr[2]])
aggr_h

array([0.00045   , 0.03255826, 0.22894781])

In [70]:
aggr_mu = np.array([(viruses_pop_vector[0]*viruses_mu[0]+viruses_pop_vector[1]*viruses_mu[1])/viruses_pop_vector_aggr[0],
          (viruses_pop_vector[2]*viruses_mu[2]+viruses_pop_vector[3]*viruses_mu[3])/viruses_pop_vector_aggr[1],
          (viruses_pop_vector[4]*viruses_mu[4]+viruses_pop_vector[5]*viruses_mu[5]+viruses_pop_vector[6]*viruses_mu[6])/viruses_pop_vector_aggr[2]])
aggr_mu

array([0.2       , 0.27555235, 0.63077265])

In [71]:
p_death = (1-aggr_p) * aggr_xi * aggr_h * aggr_mu
p_death

array([4.51228375e-06, 1.16873943e-03, 2.81312918e-02])